# Region Type Correction Patch (Revised)
This notebook provides a robust patch to correct the `Region Type` column in existing FISH-RT probe design CSV files using a **unified 1-indexed inclusive coordinate system**.

### Why is this needed?
1. **Strand Orientation**: Initial versions did not flip relative coordinates for negative strand genes.
2. **Coordinate Mismatch**: There was a 1-bp offset discrepancy between probe positions and exon boundaries.
3. **Transcript Transparency**: The pipeline now uses "longest spliced length" for transcript selection and allows overrides.

### Design Policy Applied
- **1-indexed**: Gene sequence start = Position 1.
- **Longest Spliced**: Selects transcript with the most total exon length.
- **Aggregate Labeling**: A probe is labeled `exon` if it overlaps with an exon in **any** known isoform of the gene, providing a safety margin.

In [ ]:
import pandas as pd
import os
from pathlib import Path
from gene_fetcher import GeneSequenceFetcher
from utils.sequence_utils import determine_region_type
import config
from rich.console import Console
from rich.table import Table

console = Console()

# ========================================
# CONFIGURATION
# ========================================
FILES_TO_CORRECT = [
    "FISH_RT_probes_CANDIDATES.csv",
    "FISH_RT_probes_FINAL_SELECTION.csv",
    "FISH_RT_probes_WITH_PRIMERS.csv",
]

BASE_DIR = config.FISH_RT_CONFIG.get("output_directory", ".")
OUTPUT_SUFFIX = "_CORRECTED"

# Strategy for RegionType Labeling
AGGREGATE_EXONS = True  # If True, label as 'exon' if it hits an exon in ANY transcript
# ========================================

console.print(f"🔍 [bold cyan]Initializing Gene Fetcher...[/bold cyan]")
fetcher = GeneSequenceFetcher(config.FISH_RT_CONFIG)
fetcher.aggregate_exons_for_labeling = AGGREGATE_EXONS

def process_file(file_path, output_path):
    if not Path(file_path).exists():
        console.print(f"[yellow]⚠️  Skipping: File not found at {file_path}[/yellow]")
        return
        
    console.print(f"\n[bold]📖 Processing: {file_path}[/bold]")
    df = pd.read_csv(file_path)
    
    if 'GeneName' not in df.columns or 'Probe_Start' not in df.columns:
        console.print(f"[red]❌ ERROR: Required columns missing. Skipping.[/red]")
        return

    genes_in_file = df['GeneName'].unique()
    gene_data_map = {}
    
    # Summary table for transcripts used
    table = Table(title="Transcript Design Summary")
    table.add_column("Gene", style="cyan")
    table.add_column("Transcript ID", style="magenta")
    table.add_column("Strand", justify="center")
    table.add_column("Spliced Length", justify="right")
    
    for gene in genes_in_file:
        data = fetcher.fetch_gene_sequence(gene)
        if data:
            gene_data_map[gene] = data
            table.add_row(
                gene, 
                data['transcript_id'], 
                "+" if data['strand'] == 1 else "-",
                f"{data['transcript_length']} bp"
            )
        else:
            console.print(f"[yellow]⚠️  WARNING: Could not fetch data for {gene}.[/yellow]")

    console.print(table)
    corrections_made = 0
    
    def correct_row(row):
        nonlocal corrections_made
        gene = row['GeneName']
        if gene not in gene_data_map:
            return row['RegionType']
            
        data = gene_data_map[gene]
        probe_start = row['Probe_Start']
        probe_end = row['Probe_End']
        probe_length = row['Probe_Length']
        
        # Calculate 1-indexed relative position following the RNA sense 5'->3'
        if data['strand'] == 1:
            rel_pos = probe_start - data['genomic_start'] + 1
        else:
            rel_pos = data['genomic_end'] - probe_end + 1
            
        # Classification using aggregate exons if enabled
        exons_to_check = data.get('all_known_exons', data['exon_regions'])
        
        new_region = determine_region_type(
            rel_pos, 
            rel_pos + probe_length - 1, 
            exons_to_check, 
            data['intron_regions']
        )
        
        if new_region != row['RegionType']:
            corrections_made += 1
            
        return new_region

    df['RegionType'] = df.apply(correct_row, axis=1)
    
    console.print(f"[green]✅ Correction complete! Rows updated: {corrections_made}[/green]")
    df.to_csv(output_path, index=False)
    console.print(f"💾 Saved to: {output_path}")

for filename in FILES_TO_CORRECT:
    full_path = Path(BASE_DIR) / filename
    output_path = Path(BASE_DIR) / f"{Path(filename).stem}{OUTPUT_SUFFIX}.csv"
    process_file(full_path, output_path)

console.print("\n[bold green]✨ All processing finished![/bold green]")